<div style="float:right; padding-top: 15px; padding-right: 15px">
    <div>
        <a href="https://whiteboxml.com">
            <img src="https://whiteboxml.com/static/img/logo/black_bg_white.svg" width="250">
        </a>
    </div>
</div>

# subqueries and temptables

## 1. introduction

* let's practice subqueries again (with clause...)
* views and temporary tables

## 2. sample database (publications)

In [5]:
# let's load jupyter sql extension

%load_ext sql
%config SqlMagic.autocommit = False

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [6]:
# load database

%sql sqlite:///data/publications.db

getting tables in publications database:

In [7]:
%%sql tables <<

SELECT 
    name
FROM 
    sqlite_master 
WHERE 
    type ='table' AND 
    name NOT LIKE 'sqlite_%';

 * sqlite:///data/publications.db
Done.
Returning data to local variable tables


In [8]:
tables.DataFrame()

,name
0,authors
1,discounts
2,employee
3,jobs
4,pub_info
5,publishers
6,roysched
7,sales
8,stores
9,titleauthor


## 3. subqueries

### practice with clause...

get a query to replicate our famous benchmarking table with:
* store name
* total (distinct) orders
* total (distinct) items sold
* total units sold

In [9]:
%%sql select <<

SELECT 
stores.stor_name AS Store, 
COUNT(DISTINCT(sales.ord_num)) AS Orders, 
COUNT(sales.title_id) AS Items,
SUM(sales.qty) AS Qty
FROM sales
INNER JOIN stores 
ON stores.stor_id = sales.stor_id
GROUP BY Store

 * sqlite:///data/publications.db
Done.
Returning data to local variable select


In [10]:
select.DataFrame()

,Store,Orders,Items,Qty
0,Barnum's,2,2,125
1,Bookbeat,4,4,80
2,Doc-U-Mat: Quality Laundry and Books,3,6,130
3,Eric the Read Books,2,2,8
4,Fricative Bookshop,3,3,60
5,News & Brews,2,4,90


now made a new query to retrieve:
* the average distinct books sold by order
* the average items sold by order

to compute average items per order and average quantity per item for each store... there are two ways...

In [11]:
%%sql with_example_1 <<

WITH store_total_sales AS 
(
SELECT 
stores.stor_name AS Store, 
COUNT(DISTINCT(sales.ord_num)) AS Orders, 
COUNT(sales.title_id) AS Items,
SUM(sales.qty) AS Qty
FROM sales
INNER JOIN stores 
ON stores.stor_id = sales.stor_id
GROUP BY Store
)
SELECT 
Store, 
Items/Orders as AvgItems,
Qty/Orders as AvgQty
from store_total_sales;

 * sqlite:///data/publications.db
Done.
Returning data to local variable with_example_1


In [12]:
with_example_1.DataFrame()

,Store,AvgItems,AvgQty
0,Barnum's,1,62
1,Bookbeat,1,20
2,Doc-U-Mat: Quality Laundry and Books,2,43
3,Eric the Read Books,1,4
4,Fricative Bookshop,1,20
5,News & Brews,2,45


### another with example (more complex)

let's get order details only for those stores with a ratio of distinct titles / orders > 1...

In [30]:
%%sql with_example_2 <<

WITH store_total_sales AS 
(
SELECT
stores.stor_id AS StoreID,
stores.stor_name AS Store, 
COUNT(DISTINCT(sales.ord_num)) AS Orders, 
COUNT(sales.title_id) AS Items,
SUM(sales.qty) AS Qty
FROM sales
INNER JOIN stores 
ON stores.stor_id = sales.stor_id
GROUP BY Store
)
SELECT Store, 
sales.ord_num as OrderNumber,
sales.ord_date as OrderDate,
titles.title as Title,
sales.qty as Qty,
titles.price as Price,
titles.type as Type
FROM store_total_sales
JOIN sales ON sales.stor_id = store_total_sales.StoreID
JOIN titles ON sales.title_id = titles.title_id
WHERE store_total_sales.Items / store_total_sales.Orders > 1;

 * sqlite:///data/publications.db
Done.
Returning data to local variable with_example_2


In [31]:
with_example_2.DataFrame()

,Store,OrderNumber,OrderDate,Title,Qty,Price,Type
0,Doc-U-Mat: Quality Laundry and Books,N914008,1994-09-14 00:00:00,Is Anger the Enemy?,20,10.95,psychology
1,Doc-U-Mat: Quality Laundry and Books,N914014,1994-09-14 00:00:00,The Gourmet Microwave,25,2.99,mod_cook
2,Doc-U-Mat: Quality Laundry and Books,P3087a,1993-05-29 00:00:00,Computer Phobic AND Non-Phobic Individuals: Be...,20,21.59,psychology
3,Doc-U-Mat: Quality Laundry and Books,P3087a,1993-05-29 00:00:00,Life Without Fear,25,7.00,psychology
4,Doc-U-Mat: Quality Laundry and Books,P3087a,1993-05-29 00:00:00,Prolonged Data Deprivation: Four Case Studies,15,19.99,psychology
5,Doc-U-Mat: Quality Laundry and Books,P3087a,1993-05-29 00:00:00,Emotional Security: A New Algorithm,25,7.99,psychology
6,News & Brews,D4482,1994-09-14 00:00:00,Is Anger the Enemy?,10,10.95,psychology
7,News & Brews,P2121,1992-06-15 00:00:00,"Onions, Leeks, and Garlic: Cooking Secrets of ...",40,20.95,trad_cook
8,News & Brews,P2121,1992-06-15 00:00:00,Fifty Years in Buckingham Palace Kitchens,20,11.95,trad_cook
9,News & Brews,P2121,1992-06-15 00:00:00,"Sushi, Anyone?",20,14.99,trad_cook


## 4. views

In [24]:
%%sql sql_view <<
CREATE VIEW IF NOT EXISTS orders_report AS
WITH store_total_sales AS 
(
SELECT
stores.stor_id AS StoreID,
stores.stor_name AS Store, 
COUNT(DISTINCT(sales.ord_num)) AS Orders, 
COUNT(sales.title_id) AS Items,
SUM(sales.qty) AS Qty
FROM sales
INNER JOIN stores 
ON stores.stor_id = sales.stor_id
GROUP BY Store
)
SELECT
Store, 
sales.ord_num as OrderNumber,
sales.ord_date as OrderDate,
titles.title as Title,
sales.qty as Qty,
titles.price as Price,
titles.type as Type
FROM store_total_sales
JOIN sales ON sales.stor_id = store_total_sales.StoreID
JOIN titles ON sales.title_id = titles.title_id
WHERE store_total_sales.Items / store_total_sales.Orders > 1;

 * sqlite:///data/publications.db
Done.
Returning data to local variable sql_view


action queries return empty querysets as usual...

In [21]:
sql_view.DataFrame()

""


let's see if there is a view named **orders_report**:

In [22]:
%%sql sql_view_select <<
SELECT * FROM orders_report;

 * sqlite:///data/publications.db
Done.
Returning data to local variable sql_view_select


In [23]:
sql_view_select.DataFrame()

,Store,OrderNumber,OrderDate,Title,Qty,Price,Type
0,Doc-U-Mat: Quality Laundry and Books,N914008,1994-09-14 00:00:00,Is Anger the Enemy?,20,10.95,psychology
1,Doc-U-Mat: Quality Laundry and Books,N914014,1994-09-14 00:00:00,The Gourmet Microwave,25,2.99,mod_cook
2,Doc-U-Mat: Quality Laundry and Books,P3087a,1993-05-29 00:00:00,Computer Phobic AND Non-Phobic Individuals: Be...,20,21.59,psychology
3,Doc-U-Mat: Quality Laundry and Books,P3087a,1993-05-29 00:00:00,Life Without Fear,25,7.00,psychology
4,Doc-U-Mat: Quality Laundry and Books,P3087a,1993-05-29 00:00:00,Prolonged Data Deprivation: Four Case Studies,15,19.99,psychology
5,Doc-U-Mat: Quality Laundry and Books,P3087a,1993-05-29 00:00:00,Emotional Security: A New Algorithm,25,7.99,psychology
6,News & Brews,D4482,1994-09-14 00:00:00,Is Anger the Enemy?,10,10.95,psychology
7,News & Brews,P2121,1992-06-15 00:00:00,"Onions, Leeks, and Garlic: Cooking Secrets of ...",40,20.95,trad_cook
8,News & Brews,P2121,1992-06-15 00:00:00,Fifty Years in Buckingham Palace Kitchens,20,11.95,trad_cook
9,News & Brews,P2121,1992-06-15 00:00:00,"Sushi, Anyone?",20,14.99,trad_cook


some technologies supports materialized views...
* https://www.postgresql.org/docs/current/rules-materializedviews.html

materialized views can be refreshed, because are created as a query to other tables, standard tables can not be refreshed...

## 5. temporary tables

temporary tables are valid for the current sql session... and vanish at disconnection...

In [6]:
%%sql create_temp_table <<

CREATE TEMPORARY TABLE IF NOT EXISTS store_total_sales AS
SELECT
stores.stor_id AS StoreID,
stores.stor_name AS Store, 
COUNT(DISTINCT(sales.ord_num)) AS Orders, 
COUNT(sales.title_id) AS Items,
SUM(sales.qty) AS Qty
FROM sales
INNER JOIN stores 
ON stores.stor_id = sales.stor_id
GROUP BY Store

 * sqlite:///data/publications.db
Done.
Returning data to local variable create_temp_table


In [7]:
create_temp_table.DataFrame()

""


In [8]:
%%sql query_temp_table <<

SELECT * FROM store_total_sales;

 * sqlite:///data/publications.db
Done.
Returning data to local variable query_temp_table


In [9]:
query_temp_table.DataFrame()

,StoreID,Store,Orders,Items,Qty
0,7066,Barnum's,2,2,125
1,8042,Bookbeat,4,4,80
2,7131,Doc-U-Mat: Quality Laundry and Books,3,6,130
3,6380,Eric the Read Books,2,2,8
4,7896,Fricative Bookshop,3,3,60
5,7067,News & Brews,2,4,90


this table expires after session ends...

## 6. differences between tables, views, and materialized views...

can someone explain the main differences at this point?

## 7. if there is extra time... let's do the same using pandas

let's replicate part of our analysis using pandas...

In [25]:
from sqlalchemy import create_engine
import pandas as pd

In [26]:
eng = create_engine('sqlite:///data/publications.db')

In [27]:
sales = pd.read_sql_table('sales', con=eng)
stores = pd.read_sql_table('stores', con=eng)

In [28]:
sales.head()

,stor_id,ord_num,ord_date,qty,payterms,title_id
0,6380,6871,1994-09-14,5,Net 60,BU1032
1,6380,722a,1994-09-13,3,Net 60,PS2091
2,7066,A2976,1993-05-24,50,Net 30,PC8888
3,7066,QA7442.3,1994-09-13,75,ON invoice,PS2091
4,7067,D4482,1994-09-14,10,Net 60,PS2091


In [29]:
stores.head()

,stor_id,stor_name,stor_address,city,state,zip
0,6380,Eric the Read Books,788 Catamaugus Ave.,Seattle,WA,98056
1,7066,Barnum's,567 Pasadena Ave.,Tustin,CA,92789
2,7067,News & Brews,577 First St.,Los Gatos,CA,96745
3,7131,Doc-U-Mat: Quality Laundry and Books,24-A Avogadro Way,Remulade,WA,98014
4,7896,Fricative Bookshop,89 Madison St.,Fremont,CA,90019


In [30]:
import numpy as np

In [31]:
pd.merge(sales, stores, on='stor_id').groupby('stor_id').agg({'qty': 'sum', 'ord_num': 'count'})

,qty,ord_num
stor_id,,
6380,8,2
7066,125,2
7067,90,4
7131,130,6
7896,60,3
8042,80,4


In [17]:
sales

,stor_id,ord_num,ord_date,qty,payterms,title_id
0,6380,6871,1994-09-14,5,Net 60,BU1032
1,6380,722a,1994-09-13,3,Net 60,PS2091
2,7066,A2976,1993-05-24,50,Net 30,PC8888
3,7066,QA7442.3,1994-09-13,75,ON invoice,PS2091
4,7067,D4482,1994-09-14,10,Net 60,PS2091
5,7067,P2121,1992-06-15,40,Net 30,TC3218
6,7067,P2121,1992-06-15,20,Net 30,TC4203
7,7067,P2121,1992-06-15,20,Net 30,TC7777
8,7131,N914008,1994-09-14,20,Net 30,PS2091
9,7131,N914014,1994-09-14,25,Net 30,MC3021


## 6. query optimization

https://www.postgresql.org/docs/10/geqo-intro.html

<div style="padding-top: 25px; float: right">
    <div>    
        <i>&nbsp;&nbsp;© Copyright by</i>
    </div>
    <div>
        <a href="https://whiteboxml.com">
            <img src="https://whiteboxml.com/static/img/logo/black_bg_white.svg" width="125">
        </a>
    </div>
</div>